In [1]:
from ssri_interactions.interactions.loaders import SpontaneousActivityLoader
from ssri_interactions.io import load_derived_generic
from ssri_interactions.interactions.preprocessors import InteractionsPreprocessor
from ssri_interactions.interactions.seqnmf import SeqNMF
import pandas as pd
from ssri_interactions.config import Config

/opt/miniconda3/envs/ssri/lib/python3.8/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.1, the latest is 0.5.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
neuron_types = load_derived_generic('neuron_types.csv')
sessions = neuron_types["session_name"].dropna().unique()

res = []
Ks = [6]
Ls = [10,]
bin_widths = [0.05, 0.1, 0.5, 1, 2]

loader_fac = lambda session_name, bin_width: SpontaneousActivityLoader(session_name=session_name, bin_width=bin_width, shuffle=False)
preprocessor_fac = lambda: InteractionsPreprocessor(z=False, minmax=True, gaussian_sigma=None)
model_fac = lambda K, L, Lambda: SeqNMF(K, L, Lambda)


for session in sessions:
    for k in Ks:
        for l in Ls:
            for bin_width in bin_widths:
                loader = loader_fac(session, bin_width,)
                preprocessor = preprocessor_fac()
                model = model_fac(k, l, 0.1)
                spikes = preprocessor(loader())
                score = model.sequenciness_score(spikes.T)
                res.append({
                    "session_name": session,
                    "k": k,
                    "L": l,
                    "bin_width": bin_width,
                    "score": score,
                })


In [6]:
df_out = pd.DataFrame.from_records(res)
dd = Config.derived_data_dir

df_out.to_csv(dd / "seqnmf_sequenciness_scores.csv")